#### 전통퀀트투자 전략
---
- 정량적 방법론을 기반으로 투자 의사를 결정
    - 평균 회귀
        - 볼린저 밴드 : 현재 주가가 상대적으로 높은지 낮은지 판단할 때 사용하는 지표
        - 듀얼 모멘텀 전략 : 절대 모멘텀 + 상대 모멘텀
    - 이동평균선
    - 상대 강도 지수
    - 스토캐스틱 오실레이터
- 양적 가치 투자 철학
    - 유니버스(종목)확인
    - 아웃라이어 제거
    - 가치 찾기
    - 퀄리티 찾기
    - 신념을 가지고 투자하기

조정 종가

In [ ]:
def sort_value(someValue, asc=True, standard=0): # 특정 지표를 오름차순(내림차순=False)으로 순위를 매김
    
    someValue_mask = someValue.mask(someValue<standard, np.nan)
    someValue_rank = someValue_mask.rank(axis=1, ascending=asc, na_option="bottom")
    
    return someValue_rank

def MagicFormula(PER, ROA): #마법공식
    
    PER_rank = sort_value(PER, asc=True)
    ROA_rank = sort_value(ROA, asc=False)
    
    result_rank = PER_rank + ROA_rank
    
    result = result_rank.rank(axis=1, ascending=True)
    
    result = result.where(result <= 10, 0)
    result = result.mask(result > 0, 1)
    
    return result

In [1]:
import FinanceDataReader as fdr

In [2]:
krx_df = fdr.StockListing('KRX')
krx_df.head()

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,095570,KOSPI,AJ네트웍스,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,006840,KOSPI,AK홀딩스,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,054620,KOSDAQ,APS홀딩스,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,265520,KOSDAQ,AP시스템,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [7]:
krx_df['Symbol'][0]

'060310'

In [8]:
import pandas as pd
import numpy as np


In [9]:
df = pd.read_csv('./data/PER_ROA.csv',engine='python',encoding='cp949')

In [10]:
df.head()

,종목명,현재가,전일비,등락률,액면가,거래량,시가총액,영업이익,PER,ROA
0,삼성전자,43950,100,-0.23%,100,4637815,2623719,588867.0,7.30,13.83
1,SK하이닉스,74400,600,0.81%,5000,1673056,541634,208438.0,3.49,28.49
2,삼성전자우,36350,350,-0.95%,100,553133,299119,NaN,6.03,NaN
3,현대차,126000,2000,1.61%,5000,252713,269222,24222.0,23.54,0.92
4,NAVER,148500,500,0.34%,100,367244,244748,9425.0,37.72,7.02


In [13]:
# with open으로 데이터 읽어오기

import csv
line_list = []
with open('./data/PER_ROA.csv', encoding='cp949') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',') #  ',' 구분자로 파일을 읽는다.
    for row in csv_reader:  
        if '' in row:     # 한줄씩 순회하는데 NaN이 포함된 Row는 건너뛴다.
            pass
        else : 
            line_list.append(row) # 리스트에 추가.

df = pd.DataFrame(data=line_list[1:],columns=line_list[0])
df.head()

,종목명,현재가,전일비,등락률,액면가,거래량,시가총액,영업이익,PER,ROA
0,삼성전자,43950,100,-0.23%,100,4637815,2623719,588867,7.3,13.83
1,SK하이닉스,74400,600,0.81%,5000,1673056,541634,208438,3.49,28.49
2,현대차,126000,2000,1.61%,5000,252713,269222,24222,23.54,0.92
3,NAVER,148500,500,0.34%,100,367244,244748,9425,37.72,7.02
4,LG화학,329500,9500,2.97%,5000,176590,232602,22461,17.52,5.63


In [14]:
def sort_value( s_value, asc = True, standard = 0): 
    '''
    description
        특정 지표값을 정렬한다.
        
    parameters
        s_value : pandas Series
            정렬할 데이터를 받는다.
            
        asc : bool
            True : 오름차순
            False : 내림차순
            
        standard : int
            조건에 맞는 값을 True로 대체하기 위한 기준값 
    returns 
        s_value_mask_rank : pandas Series
            정렬된 순위
    ''' 
    
    s_value_mask = s_value.mask(s_value < standard, np.nan)  #지표별 기준값 미만은 필터링 한다.
    s_value_mask_rank = s_value_mask.rank( ascending=asc, na_option="bottom") # 필터링된 종목에서 순위 선정
    
    return s_value_mask_rank

In [15]:
per = pd.to_numeric(df['PER'])
roa = pd.to_numeric(df['ROA'])
# per = df['PER']
per_rank = sort_value(per, asc=True, standard=0 ) # PER 지표값을 기준으로 순위 정렬 및 0 미만 값 제거
roa_rank = sort_value(roa, asc=False, standard=0 )# ROA 지표값을 기준으로 순위 정렬 및 0 미만 값 제거

In [16]:
type(per_rank)

pandas.core.series.Series

In [17]:
per_rank.head()

0    139.0
1     22.0
2    424.0
3    487.0
4    367.0
Name: PER, dtype: float64

In [18]:
roa_rank.head()

0     18.0
1      5.0
2    478.5
3    103.0
4    149.0
Name: ROA, dtype: float64

In [19]:
result_rank = per_rank + roa_rank   # PER 순위 ROA 순위 합산
result_rank = sort_value(result_rank, asc=True)  # 합산 순위 정렬
result_rank = result_rank.where(result_rank <= 10, 0)  # 합산 순위 필터링
result_rank = result_rank.mask(result_rank > 0, 1)  # 순위 제거

In [20]:
result_rank.head()

0    0.0
1    1.0
2    0.0
3    0.0
4    0.0
dtype: float64

In [21]:
result_rank.sum()

10.0

In [22]:
mf_df = df.loc[result_rank > 0,['종목명','시가총액']].copy() # 선택된 종목 데이터프레임 복사
mf_stock_list = df.loc[result_rank > 0, '종목명'].values # 선택된 종목명 추출

In [23]:
mf_df

,종목명,시가총액
1,SK하이닉스,541634
100,효성,18353
175,대한유화,7508
176,HDC,7468
310,한일홀딩스,2904
322,신대양제지,2756
372,세아제강지주,2044
394,SIMPAC,1835
408,케이씨,1694
726,에쓰씨엔지니어링,354


In [24]:
mf_stock_list

array(['SK하이닉스', '효성', '대한유화', 'HDC', '한일홀딩스', '신대양제지', '세아제강지주',
       'SIMPAC', '케이씨', '에쓰씨엔지니어링'], dtype=object)

In [25]:
mf_df['종목코드'] = ''
for stock in mf_stock_list :
    # 데이터프레임에서 조건에 맞는 데이터만 불러온다.
    mf_df.loc[mf_df['종목명'] == stock,'종목코드'] = krx_df[krx_df['Name'] == stock]['Symbol'].values

In [26]:
mf_df

,종목명,시가총액,종목코드
1,SK하이닉스,541634,000660
100,효성,18353,004800
175,대한유화,7508,006650
176,HDC,7468,012630
310,한일홀딩스,2904,003300
322,신대양제지,2756,016590
372,세아제강지주,2044,003030
394,SIMPAC,1835,009160
408,케이씨,1694,029460
726,에쓰씨엔지니어링,354,023960


In [27]:
# 수익률 구하는 방식1
mf_df['2019_수익률'] = ''
for x in mf_df['종목코드'].values : 
    df = fdr.DataReader(x, '2019-01-01','2019-12-31') # 개별 종목 가격 데이터 호출
    cum_ret = df.loc[df.index[-1], 'Close'] / df.loc[df.index[0],'Close'] -1  # 2019년도 누적 수익률 계산
    mf_df.loc[mf_df['종목코드'] == x, '2019_수익률' ] = cum_ret  # 누적 수익률 저장
    df = None

In [28]:
mf_df

,종목명,시가총액,종목코드,2019_수익률
1,SK하이닉스,541634,000660,0.552805
100,효성,18353,004800,0.649635
175,대한유화,7508,006650,-0.20339
176,HDC,7468,012630,-0.332326
310,한일홀딩스,2904,003300,-0.197424
322,신대양제지,2756,016590,0.034783
372,세아제강지주,2044,003030,-0.008368
394,SIMPAC,1835,009160,0.206967
408,케이씨,1694,029460,0.594017
726,에쓰씨엔지니어링,354,023960,-0.062092


In [29]:
# 수익률 구하는 방식2
mf_df_rtn = pd.DataFrame()
for x in mf_df['종목코드'].values : 
#     print(x ,', ' , mf_df.loc[mf_df['종목코드'] == x, '종목명' ].values[0])
    df = fdr.DataReader(x, '2019-01-01','2019-12-31') # 개별 종목 가격 데이터 호출
    df['daily_rtn'] = df['Close'].pct_change(periods=1)
    df['cum_rtn'] = (1+df['daily_rtn']).cumprod()
    cum_ret = df.loc[df.index[-1],'cum_rtn']
    mf_df.loc[mf_df['종목코드'] == x, '2019_수익률' ] = cum_ret  # 누적 수익률 저장
    df = None

In [30]:
mf_df

,종목명,시가총액,종목코드,2019_수익률
1,SK하이닉스,541634,000660,1.552805
100,효성,18353,004800,1.649635
175,대한유화,7508,006650,0.79661
176,HDC,7468,012630,0.667674
310,한일홀딩스,2904,003300,0.802576
322,신대양제지,2756,016590,1.034783
372,세아제강지주,2044,003030,0.991632
394,SIMPAC,1835,009160,1.206967
408,케이씨,1694,029460,1.594017
726,에쓰씨엔지니어링,354,023960,0.937908


In [31]:
mf_df_rtn = pd.DataFrame()

for x in mf_df['종목코드'].values : 
#     print(x ,', ' , mf_df.loc[mf_df['종목코드'] == x, '종목명' ].values[0])
    df = fdr.DataReader(x, '2019-01-01','2019-12-31') # 개별 종목 가격 데이터 호출
    df['daily_rtn'] = df['Close'].pct_change(periods=1)
    df['cum_rtn'] = (1+df['daily_rtn']).cumprod()
    cum_ret = df.loc[df.index[-1],'cum_rtn']
    mf_df.loc[mf_df['종목코드'] == x, '2019_수익률' ] = cum_ret  # 누적 수익률 저장
    df = None

In [32]:
mf_df

,종목명,시가총액,종목코드,2019_수익률
1,SK하이닉스,541634,000660,1.552805
100,효성,18353,004800,1.649635
175,대한유화,7508,006650,0.79661
176,HDC,7468,012630,0.667674
310,한일홀딩스,2904,003300,0.802576
322,신대양제지,2756,016590,1.034783
372,세아제강지주,2044,003030,0.991632
394,SIMPAC,1835,009160,1.206967
408,케이씨,1694,029460,1.594017
726,에쓰씨엔지니어링,354,023960,0.937908


In [33]:
# 수익률 구하는 방식3
for ind,val in enumerate(mf_df['종목코드'].values) : 
    # 가독성을 위해 종목명 추출.
    code_name = mf_df.loc[mf_df['종목코드'] == val,'종목명'].values[0]
    print(val, code_name)
    df = fdr.DataReader(val, '2019-01-01','2019-12-31') # 개별 종목 가격 데이터 호출
    if ind == 0 :
        mf_df_rtn = pd.DataFrame(index=df.index) # 첫 번째 종목코드 인덱스 활용한 데이터프레임 생성
    df['daily_rtn'] = df['Close'].pct_change(periods=1) # period 기간 차이만큼 변동율 계산
    df['cum_rtn'] = (1+df['daily_rtn']).cumprod() # 누적 곱 계산
    tmp = df.loc[:,['cum_rtn']].rename(columns={'cum_rtn':code_name}) # 가독성을 위한 컬럼명 변경
    mf_df_rtn = mf_df_rtn.join(tmp,how='left') # 새로 계산된 누적 수익률 추가.
    df = None # 데이터 프레임 초기화

000660 SK하이닉스
004800 효성
006650 대한유화
012630 HDC
003300 한일홀딩스
016590 신대양제지
003030 세아제강지주
009160 SIMPAC
029460 케이씨
023960 에쓰씨엔지니어링


In [34]:
mf_df_rtn.tail()

,SK하이닉스,효성,대한유화,HDC,한일홀딩스,신대양제지,세아제강지주,SIMPAC,케이씨,에쓰씨엔지니어링
Date,,,,,,,,,,
2019-12-23,1.561056,1.722628,0.820339,0.655589,0.814385,1.041739,0.974895,1.192623,1.337607,0.947712
2019-12-24,1.547855,1.747654,0.820339,0.655589,0.804333,1.026087,0.990586,1.178279,1.354701,0.908497
2019-12-26,1.564356,1.758081,0.823729,0.667674,0.820826,1.045217,1.003138,1.192623,1.410256,0.928105
2019-12-27,1.584158,1.660063,0.803390,0.664653,0.800722,1.041739,0.991632,1.200820,1.534188,0.911765
2019-12-30,1.552805,1.649635,0.796610,0.667674,0.802576,1.034783,0.991632,1.206967,1.594017,0.937908
